<a href="https://colab.research.google.com/github/bluezdot/ShopeeImageClassification/blob/main/ShopeeImagePreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bluezdot/ShopeeImageClassification.git

fatal: destination path 'ShopeeImageClassification' already exists and is not an empty directory.


In [2]:
!pip install albumentations

In [10]:
%cd ShopeeImageClassification/dataset/

/content/ShopeeImageClassification/dataset


In [3]:
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
import albumentations as A
import albumentations.pytorch
from albumentations.pytorch import ToTensorV2

ImportError: ignored

In [14]:
train_img_path = './train_images'
test_img_path = './test_images'
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [6]:
def my_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [17]:
# drop duplicate by p-hash
train_df = train_df.drop_duplicates(subset=['image_phash'],keep = 'first')
train_df.image_phash.value_counts()

a16bc292ed094bf6    1
bc4a94ed5f2fa480    1
ca0e2a7495f1d84f    1
ec6bd9b5a74488c8    1
dbccc786ce3018c7    1
                   ..
ed618f98930e784b    1
e59a9a659e61329a    1
f5e596677098ca24    1
bf30e336620f708b    1
f876d2c887a986ca    1
Name: image_phash, Length: 28735, dtype: int64

In [18]:
# check if label group preserved
train_df.label_group.value_counts()

1141798720    44
159351600     43
1091404026    39
3489985175    37
562358068     36
              ..
370710977      1
3601891778     1
2067677642     1
2213950251     1
1332066608     1
Name: label_group, Length: 11004, dtype: int64

In [19]:
train_df.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [20]:
class ShopeeDataset(Dataset):

    def __init__(self, dataframe, dir_path,transforms):
        self.dataframe = dataframe
        self.dir_path = dir_path
        self.transforms = transforms

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        img_info = self.dataframe.iloc[index]
        
        img = cv2.imread(self.dir_path + '/' + img_info['image'])
        label = torch.tensor([img_info['label_group']])

        if self.transforms:
            augmented = self.transforms(image=img) 
            img = augmented['image']

        return img, label

albumentations_transforms = albumentations.Compose([
    albumentations.Resize(300, 300, interpolation = cv2.INTER_LANCZOS4), 
    albumentations.Sharpen(),
    albumentations.ColorJitter(brightness=0.1, contrast=0.4, saturation=0, hue=0),
    albumentations.ShiftScaleRotate(rotate_limit=180, p=0.5, interpolation = cv2.INTER_LANCZOS4),
    albumentations.Normalize(mean=(0, 0, 0), std=(1, 1, 1)),
    ToTensorV2() 
])
train_ds = ShopeeDataset(train_df, train_img_path, albumentations_transforms)

my_seed(33)

train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
# visualize a batch
num_samples = 8
fig, ax = plt.subplots(1, num_samples, figsize=(25, 10))
for i in range(num_samples):
    ax[i].imshow((train_ds[32][0]).permute(1, 2, 0))
    ax[i].axis('off')
#i = iter(train_dl)
#plt.figure(figsize=(20, 5))
#j = 1
#for im in next(i)[0]:
#    plt.subplot(1, 4, j)
#    j += 1
#    plt.imshow(im.permute(1, 2, 0))

AttributeError: ignored